In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13081,2024-03-19,Eua Nba,22:00,Minnesota Timberwolves,Denver Nuggets,2.26,1.54,212.5,1.89,1.97,1.5,2.05,1.59,8dtO7NA1,0.442478,0.649351,0.529101,0.507614,0.091829,0.4,0.2,NaN,NaN,178.010,19.775126,0.111090,180.870,38.403672,0.212327,151.604,190.258,169.84,171.15,0.0,0.0,0.0,0.0,0.267956,0.029310,0.178719,-3.68,-0.736,-1.711957,0.688487,0.4,-0.288487,-1.67,-0.334,-1.616766,0.595489,0.5,-0.095489
13082,2024-03-19,Europa Euroliga,14:30,Anadolu Efes,Olympiakos,2.05,1.80,157.5,1.91,1.91,1.5,1.91,1.63,EBdrv4g3,0.487805,0.555556,0.523560,0.523560,0.043360,0.6,0.6,NaN,NaN,138.740,24.751940,0.178405,229.226,205.552059,0.896722,153.128,248.146,164.59,167.20,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,1.49,0.298,3.523490,0.589337,0.8,0.210663,-1.29,-0.258,-3.100775,0.490850,0.5,0.009150
13083,2024-03-19,Europa Euroliga,15:00,Zalgiris Kaunas,Virtus Bologna,1.63,2.38,157.5,1.91,1.91,-4.5,2.00,3.25,MydnwO89,0.613497,0.420168,0.523560,0.523560,0.033665,0.6,0.4,NaN,NaN,182.024,39.459748,0.216783,187.640,29.867960,0.159177,169.710,181.216,162.64,206.91,0.0,0.0,0.0,0.0,0.264504,0.000000,0.336718,2.72,0.544,1.158088,0.541162,0.5,-0.041162,-3.49,-0.698,-1.977077,0.421186,0.2,-0.221186
13084,2024-03-19,Eua Ncaa,19:40,Howard,Wagner,1.63,2.38,127.5,1.80,1.95,-4.5,2.05,3.30,URWs1DcQ,0.613497,0.420168,0.555556,0.512821,0.033665,0.2,0.6,NaN,NaN,127.688,41.992976,0.328872,123.206,29.293926,0.237764,100.480,137.702,89.76,106.96,0.0,0.0,0.0,0.0,0.264504,0.056569,0.330424,-2.23,-0.446,-1.412556,0.633980,0.4,-0.233980,-1.21,-0.242,-5.702479,0.502819,0.5,-0.002819
13085,2024-03-19,Eua Ncaa,22:10,Virginia,Colorado State,2.28,1.67,120.5,1.91,1.91,1.5,2.00,1.50,lCDEeiLm,0.438596,0.598802,0.523560,0.523560,0.037399,0.4,0.2,NaN,NaN,98.822,25.312655,0.256144,157.232,61.843671,0.393328,84.592,168.378,84.77,132.98,0.0,0.0,0.0,0.0,0.218398,0.000000,0.202031,0.83,0.166,7.710843,0.784344,0.9,0.115656,-1.30,-0.260,-2.576923,0.509671,0.3,-0.209671
13086,2024-03-19,Eua Nit,20:00,Georgia,Xavier,1.83,2.00,151.5,1.80,1.95,-2.5,2.00,2.30,x82mZc99,0.546448,0.500000,0.555556,0.512821,0.046448,0.2,0.8,NaN,NaN,148.740,37.944849,0.255109,135.698,23.942197,0.176437,127.336,215.982,106.88,142.12,0.0,0.0,0.0,0.0,0.062772,0.056569,0.098666,-1.75,-0.350,-2.371429,0.698918,0.7,0.001082,-2.04,-0.408,-2.450980,0.435569,0.4,-0.035569
13087,2024-03-19,Eua Nit,20:00,LSU,North Texas,1.69,2.22,135.5,1.80,1.95,-3.5,2.00,2.90,Us3qzvf3,0.591716,0.450450,0.555556,0.512821,0.042166,0.6,0.4,NaN,NaN,226.342,108.145065,0.477795,142.718,30.519837,0.213847,162.354,160.122,109.21,102.95,0.0,0.0,0.0,0.0,0.191696,0.056569,0.259754,-0.17,-0.034,-20.294118,0.697457,0.7,0.002543,-2.04,-0.408,-2.990196,0.473741,0.4,-0.073741
13088,2024-03-19,Europa Liga Dos Campeões,14:30,Peristeri,Tenerife,2.16,1.68,157.5,1.84,1.88,1.5,1.99,1.52,rPk6sMGQ,0.462963,0.595238,0.543478,0.531915,0.058201,0.2,0.6,NaN,NaN,195.950,76.192032,0.388834,135.112,46.576132,0.344722,142.622,138.922,126.92,132.98,0.0,0.0,0.0,0.0,0.176777,0.015207,0.189368,0.62,0.124,9.354839,0.000000,0.0,0.000000,-1.61,-0.322,-2.111801,0.000000,0.0,0.000000
13089,2024-03-19,Europa Liga Dos Campeões,16:30,Strasbourg,Tofas,1.88,1.89,162.5,1.87,1.87,-1.5,1.99,1.99,tYOvIGbd,0.531915,0.529101,0.534759,0.534759,0.061015,0.0,0.4,NaN,NaN,167.554,37.779247,0.225475,179.518,119.489872,0.665615,0.000,287.834,212.80,392.00,0.0,0.0,0.0,0.0,0.003751,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.00000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:00,Europa Euroliga,Zalgiris Kaunas,Virtus Bologna,157.5,1.91,1.0000,Over
1,19:40,Eua Ncaa,Howard,Wagner,127.5,1.80,1.0000,Over
2,20:00,Eua Nit,Georgia,Xavier,151.5,1.80,1.0000,Over
3,20:00,Eua Nit,LSU,North Texas,135.5,1.80,1.0000,Over
4,16:30,Europa Liga Dos Campeões,Strasbourg,Tofas,162.5,1.87,1.0000,Over
5,14:30,Israel Liga Leumit,Ramat Hasharon,Safed,161.5,1.85,1.0000,Over
6,14:30,Chipre Divisão A,Etha Encomi,Apop Paphou,157.5,1.87,0.9847,Over
7,22:00,Eua Nit,Butler,Minnesota,147.5,1.80,1.0000,Over
